In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/

In [ ]:
import os
import re
import json
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from src.utils.accounts import sanitize_account_name
from src.utils.innkeepr_api import call_api_with_service_token, send_to_innkeepr_api_paginated
from src.utils.constants import return_api_url_innkeepr
from src.utils.github_connection import read_yaml_from_github
from src.utils.databricks_pp import handle_databricks_cost
from src.utils.azure_pp import handle_azure_costs
from src.utils.aws_pp import handle_aws_costs
from src.utils.stackit_pp import stack_pp
from src.utils.cost_handling import return_cost_per_run

# Define Variables

In [3]:
from_date = "2024-01-01"
to_date="2025-06-23"
timestamp = "2025-06-23 13:23:56.520433"
path_to_dir = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/{timestamp}/"
path_to_data=f"{path_to_dir}final_costs_with_azure_and_aws_and_db_{from_date}_{to_date}.csv"
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/targeting_and_retraining/"
url = return_api_url_innkeepr()
stackit_cost_handling = {
    # "start": "2024-11-11",
    "exlude_date_ranges": [
        {
            "start": "2024-12-22",
            "end" : "2025-01-03",
        }
    ],
    "start":"2025-02-09" # davor keine eindeutige Kostenzuordnung zw. azure und stackit möglich in den Zeitspannen: 12.11.24 - 19.11.24, 20.12.24-21.12.24, 04.01.25-06.01.25, 10.01.25-12.01.25, 07.02.25 - 08.02.25
}

In [4]:
os.makedirs(path_to_save, exist_ok=True)

# Load Data

In [5]:
df_orignal = pd.read_csv(path_to_data)
df_orignal = df_orignal[[col for col in df_orignal.columns if "Unnamed" not in col]]
df_orignal

In [6]:
df = stack_pp(df_orignal)

In [7]:
df.info()

In [8]:
df["Prefect_Deployments"] = df["Prefect_Deployments"].replace("retrainng", "retraining")
df["Prefect_Deployments"].value_counts()

# Filter Data
Filter data for targeting runs only using Deployments and Prefect_Deployments. Valid strings are:

- targeting
- retraining
- googleConversionUpdate (smart bidding)

In [9]:
# get stackIT costs via Prefect_Deployments
targeting_and_retraining_runs = df[(df["Deployments"]=="targeting")|(df["Prefect_Deployments"]=="targeting")|(df["Deployments"]=="retraining")|(df["Prefect_Deployments"]=="retraining")|(df["Deployments"]=="googleConversionUpdate")|(df["Prefect_Deployments"]=="googleConversionUpdate")]
# Bug in cost extractor: "retrainng" instead of "retraining" (behoben)
targeting_and_retraining_runs

In [10]:
targeting_and_retraining_runs["Prefect_Deployments"].value_counts()

# Historical count of targeting runs and prefect runs

In [11]:
targeting_runs = targeting_and_retraining_runs[targeting_and_retraining_runs["Prefect_Deployments"]=="targeting"]
retraining_runs = targeting_and_retraining_runs[targeting_and_retraining_runs["Prefect_Deployments"]=="retraining"]
runs_vc_by_date = targeting_runs.groupby("date")["Prefect_Deployments"].value_counts()
runs_vc_by_date = pd.DataFrame(runs_vc_by_date).reset_index().rename(columns={"count":"count targeting runs"})
runs_vc_by_date_retraining = retraining_runs.groupby("date")["Prefect_Deployments"].value_counts()
runs_vc_by_date_retraining = pd.DataFrame(runs_vc_by_date_retraining).reset_index().rename(columns={"count":"count retraining runs"})
audiences_unique_by_date = targeting_runs.groupby("date")["audience_id"].nunique()
audiences_unique_by_date = pd.DataFrame(audiences_unique_by_date).reset_index()
concat = pd.merge(runs_vc_by_date, audiences_unique_by_date, on="date")
fig = plt.figure(figsize=(20,8))
sns.lineplot(
    x="date",
    y="count targeting runs",
    data=runs_vc_by_date[runs_vc_by_date["date"] > "2025-01-01"],
    marker="o",
    linestyle="--",
    label="# targeting runs"
)
sns.lineplot(
    x="date",
    y="count retraining runs",
    data=runs_vc_by_date_retraining[runs_vc_by_date_retraining["date"] > "2025-01-01"],
    marker="o",
    linestyle="--",
    label="# retraining runs"
)
sns.lineplot(
    x="date",
    y="audience_id",
    data=audiences_unique_by_date[audiences_unique_by_date["date"] > "2025-01-01"],
    marker="o",
    linestyle=":",
    label="unique audiences"
)
plt.title("Count Daily targeting runs")
plt.xticks(rotation=90)
plt.grid(True)

# Check Data Completion for node types, date and costs

In [12]:
print(len(targeting_and_retraining_runs))
null_values = targeting_and_retraining_runs[["node_name","date","charge","machine.type","audience_id","duration","total_charge_of_serviceName","audience"]].isnull().sum()
null_values = pd.DataFrame(null_values).rename(columns={0:"isnull"})
null_values["percentage_of_isnull"] = null_values["isnull"]/len(targeting_and_retraining_runs) * 100
null_values.sort_values(by="isnull")

## Handling Missing Data
- fehlend node_names mit existing node_names anreichern via targeting and retraining audience
- fehlende Kosten über node_names und runtime anreichern
- charges per targeting run erst ab dm 26.05.2025 via prefect_logs


## Query models for all active accounts
All past models are queried for the existing tenants in the dataset. 

In [13]:
# query models to map audience node_name with targetingOutlook (is one of the main parameter to choose the node size)
try:
    models = pd.read_csv(f"{path_to_save}all_models.csv")
    with open(f"{path_to_save}ignore_tenants.json", "r") as f:
        ignore_tenants = json.load(f)
except FileNotFoundError:
    print("query data")
    models=pd.DataFrame()
    #min_date = targeting_and_retraining_runs["date"].min()
    #min_date = (pd.to_datetime(min_date)-timedelta(days=60)).strftime("%Y-%m-%d")
    ignore_tenants = []
    accounts = call_api_with_service_token(f"{url}/core/accounts/query", {}, logging)
    for tenant in targeting_and_retraining_runs["tenant"].unique():
        print(tenant)
        account_id = [acc["id"] for acc in accounts if sanitize_account_name(acc["name"])==tenant]
        if len(account_id) > 1:
            raise Exception(f"More than one account with name {tenant}")
        if len(account_id) == 0:
            print(f"Tenant {tenant} not found in accounts")
            ignore_tenants.append(tenant)
            continue
        account_id = account_id[0]
        temp_models = send_to_innkeepr_api_paginated(
            f"{url}/models/query",
            account_id,
            {},
            logging
        )
        temp_models = pd.json_normalize(temp_models)
        if len(temp_models) == 0:
            print(f"No models found for tenant {tenant}")
            ignore_tenants.append(tenant)
            continue
        #temp_models = temp_models[temp_models["created"]>=min_date]
        models = pd.concat([models, temp_models])
    print(models.shape)
    missing_audiences = targeting_runs[targeting_runs["tenant"].isin(ignore_tenants)==False]
    missing_audiences = missing_audiences[missing_audiences["audience_id"].isin(models["audience"].unique())==False]
    if missing_audiences.empty == False:
        print(f"Missing {len(missing_audiences)} models")
        print(missing_audiences[["tenant","audience_id"]].drop_duplicates())
    models.to_csv(f"{path_to_save}all_models.csv")
    with open(f"{path_to_save}ignore_tenants.json", "w") as f:
        json.dump(list(ignore_tenants), f)
models

In [14]:
audiences_in_models_but_not_in_targeting_runs = models[
    (models["audience"].isin(targeting_runs[targeting_runs["tenant"].isin(ignore_tenants)==False]["audience_id"].unique())==False) &
    (models["created"]>=targeting_runs["date"].min())
    ]
if audiences_in_models_but_not_in_targeting_runs.empty == False:
    print(f"Found {len(audiences_in_models_but_not_in_targeting_runs)} models that are not in the targeting runs")
    raise Exception(audiences_in_models_but_not_in_targeting_runs[["audience","path","created"]])

## Merge Models and Targeting Runs by considering date and targetingOutlookDays

In [15]:
models = models[["audience","created","targetingOutlookDays"]]
models.head()

In [16]:
targeting_and_retraining_runs["tenant"] = np.where(
    targeting_and_retraining_runs["tenant"].isnull(),
    targeting_and_retraining_runs["account"].str.replace(" ","").str.lower(),
    targeting_and_retraining_runs["tenant"]
)

In [17]:
models = models.rename(columns={"audience":"audience_id","created":"created_data_model_for_testing"})
# Filter data_model_for_testing to get the most recent model for each audience before the date in data_for_testing
print(f"targeting_and_retraining_runs = {len(targeting_and_retraining_runs)}")
merged_targeting_runs_with_models = pd.merge(targeting_and_retraining_runs, models, on=['audience_id'], how="left")#suffixes=('_data_for_testing', '_data_model_for_testing'))
# macht data where the model.created < data.timestamp
merged_targeting_runs_with_models_with_previous_models = merged_targeting_runs_with_models[pd.to_datetime(merged_targeting_runs_with_models['created_data_model_for_testing'],utc=True) <= pd.to_datetime(merged_targeting_runs_with_models['timestamp'],utc=True)]
merged_targeting_runs_with_models_with_previous_models = merged_targeting_runs_with_models_with_previous_models.sort_values(by='created_data_model_for_testing', ascending=False).drop_duplicates(subset=['Prefect_Deployments','tenant','audience_id','timestamp','node_name'], keep='first')
print(f"merged_targeting_runs_with_models_with_previous_models = {len(merged_targeting_runs_with_models_with_previous_models)}")
# get data where data.timestamp does not have a matching model
merged_targeting_runs_with_models_without_previous_models = models.groupby("audience_id")["created_data_model_for_testing"].min()
merged_targeting_runs_with_models_without_previous_models = pd.DataFrame(merged_targeting_runs_with_models_without_previous_models).reset_index()
merged_targeting_runs_with_models_without_previous_models = pd.merge(targeting_and_retraining_runs, merged_targeting_runs_with_models_without_previous_models, on="audience_id")
merged_targeting_runs_with_models_without_previous_models = merged_targeting_runs_with_models_without_previous_models[pd.to_datetime(merged_targeting_runs_with_models_without_previous_models['timestamp'],utc=True) <= pd.to_datetime(merged_targeting_runs_with_models_without_previous_models['created_data_model_for_testing'],utc=True)]
print(f"merged_targeting_runs_with_models_without_previous_models = {len(merged_targeting_runs_with_models_without_previous_models)}")
# get data with null models
considered_audiences = merged_targeting_runs_with_models_with_previous_models["audience_id"].unique().tolist() + merged_targeting_runs_with_models_without_previous_models["audience_id"].unique().tolist()
merged_targeting_runs_with_models_null_models = merged_targeting_runs_with_models[
    (merged_targeting_runs_with_models["audience_id"].isin(considered_audiences) == False)]
merged_targeting_runs_with_models_null_models = merged_targeting_runs_with_models_null_models[merged_targeting_runs_with_models_null_models["tenant"].isin(ignore_tenants)==False]
print(f"merged_targeting_runs_with_models_null_models = {len(merged_targeting_runs_with_models_null_models)}")
# concate data
merged_targeting_runs_with_models = pd.concat([merged_targeting_runs_with_models_with_previous_models, merged_targeting_runs_with_models_without_previous_models,merged_targeting_runs_with_models_null_models])
merged_targeting_runs_with_models = merged_targeting_runs_with_models.reset_index(drop=True)
print(f"merged_targeting_runs_with_models = {len(merged_targeting_runs_with_models)}")
targeting_runs_to_compare = targeting_and_retraining_runs[targeting_and_retraining_runs["tenant"].isin(ignore_tenants)==False]
if len(merged_targeting_runs_with_models) != len(targeting_runs_to_compare):
    raise Exception(f"Unequal length targeting_runs_to_compare {len(targeting_runs_to_compare)} vs. merged {len(merged_targeting_runs_with_models)}")
merged_targeting_runs_with_models[["tenant","audience_id","date","node_name","timestamp","created_data_model_for_testing","targetingOutlookDays"]]

In [18]:
# if dataframe does not match lenght - check why
vc_before = pd.DataFrame(targeting_and_retraining_runs[targeting_and_retraining_runs["tenant"].isin(ignore_tenants)==False].groupby(by=["Prefect_Deployments","audience_id"])["date"].value_counts()).reset_index().sort_values(by=["audience_id","date"], ascending=False).reset_index(drop=True)
vc_after = pd.DataFrame(merged_targeting_runs_with_models.groupby(by=["Prefect_Deployments","audience_id"])["date"].value_counts()).reset_index().sort_values(by=["audience_id","date"], ascending=False).reset_index(drop=True)
if vc_after.equals(vc_before):
    print("fine")
else:
    print("not fine")

# Extract node gb size and cpu

In [19]:
def extract_node_parameters(string,node_spec=None):
    #print(string)
    if isinstance(string, str) == False:
        #print(f"String is not given")
        return None
    if "medium32g" in string:
        if node_spec == "cpu":
            return None
        elif node_spec == "gb":
            return 32
        else:
            raise ValueError(f"Node spec needs to be cpu or gb")
    if "x2large" in string:
        if node_spec == "cpu":
            return 29
        elif node_spec == "gb":
            return 350
        else:
            raise ValueError(f"Node spec needs to be cpu or gb")
    if "small" in string:
        if node_spec == "cpu":
            return 3
        elif node_spec == "gb":
            return 4
        else:
            raise ValueError(f"Node spec needs to be cpu or gb")
    if "xlarge" in string:
        if node_spec == "cpu":
            return 7
        elif node_spec == "gb":
            return 110
        else:
            raise ValueError(f"Node spec needs to be cpu or gb")
    if "medium64g" in string:
        if node_spec == "cpu":
            return 3
        elif node_spec == "gb":
            return 55
        else:
            raise ValueError(f"Node spec needs to be cpu or gb")
    if "x4llarge" in string:
        if node_spec == "cpu":
            return 29
        elif node_spec == "gb":
            return 350
        else:
            raise ValueError(f"Node spec needs to be cpu or gb")
    # match the string for cpu and gb
    match = re.search(r"(\d+)cpu(\d+)gib", string)
    if match is False:
        print(f"No match found for {string}")
        return None
    elif node_spec is None:
        raise ValueError(f"Node spec needs to be cpu or gb")
    elif node_spec == "cpu":
        return int(match.group(1))
    elif node_spec == "gb":
        return int(match.group(2))
    else:
        raise ValueError(f"Node spec needs to be cpu or gb")


In [20]:
merged_targeting_runs_with_models["node_gb"] = merged_targeting_runs_with_models["node_name"].apply(lambda x: extract_node_parameters(x,node_spec="gb"))
merged_targeting_runs_with_models["node_cpu"] = merged_targeting_runs_with_models["node_name"].apply(lambda x: extract_node_parameters(x,node_spec="cpu"))
merged_targeting_runs_with_models[["tenant","audience_id","node_name","timestamp","targetingOutlookDays","node_gb","node_cpu"]]

## Add missing node sizes via current node size mapping
- extract all known node_names and check if it can be mapped to unknown node_names

In [21]:
node_size_mapping_with_size = merged_targeting_runs_with_models.dropna(subset=["node_name"]).drop_duplicates(subset=["Prefect_Deployments","audience_id","node_name","targetingOutlookDays","node_gb","node_cpu"])[["Prefect_Deployments","tenant","audience_id","node_name","node_gb","node_cpu","targetingOutlookDays"]]
vc = pd.DataFrame(node_size_mapping_with_size.groupby("audience_id")["node_name"].nunique()).rename(columns={"node_name":"node_count"})
node_size_mapping_with_size = pd.merge(node_size_mapping_with_size, vc, on="audience_id", how="left")
vc_max = merged_targeting_runs_with_models.groupby(by=["Prefect_Deployments","audience_id","node_name","targetingOutlookDays","node_gb","node_cpu"])["timestamp"].max().reset_index()
node_size_mapping_with_size = pd.merge(node_size_mapping_with_size, vc_max, on=["Prefect_Deployments","audience_id","node_name","targetingOutlookDays","node_gb","node_cpu"], how="left")
for col in node_size_mapping_with_size.columns:
   node_size_mapping_with_size = node_size_mapping_with_size.rename(columns={col: f"{col}_nm"})
node_size_mapping_with_size.sort_values(by=["tenant_nm","audience_id_nm","Prefect_Deployments_nm"])

In [22]:
merged_targeting_runs_with_models_with_node_size = pd.merge(
    merged_targeting_runs_with_models,
    node_size_mapping_with_size,
    left_on=["Prefect_Deployments","tenant","targetingOutlookDays"],
    right_on = ["Prefect_Deployments_nm","tenant_nm","targetingOutlookDays_nm"],
    how="left"
)
merged_targeting_runs_with_models_with_node_size[["Prefect_Deployments","tenant","timestamp","audience_id","targetingOutlookDays","node_name","node_gb","node_cpu"]+node_size_mapping_with_size.columns.tolist()].sort_values(by=["tenant","audience_id","Prefect_Deployments"])

In [23]:
merged_targeting_runs_with_models_with_node_size = merged_targeting_runs_with_models_with_node_size.sort_values(by=["Prefect_Deployments","tenant","audience_id","timestamp","node_gb_nm"]).drop_duplicates(subset=["Prefect_Deployments","tenant","audience_id","timestamp"],keep="last")
merged_targeting_runs_with_models_with_node_size["merged_node_name"] = np.where(
    merged_targeting_runs_with_models_with_node_size["node_name"].isnull(),
    merged_targeting_runs_with_models_with_node_size["node_name_nm"],
    merged_targeting_runs_with_models_with_node_size["node_name"]
)
merged_targeting_runs_with_models_with_node_size[["Prefect_Deployments","tenant","timestamp","audience_id","timestamp","targetingOutlookDays","node_name","node_gb_nm","merged_node_name"]].sort_values(by=["tenant","audience_id","Prefect_Deployments"])

In [24]:
github_node_sizes_retraining = read_yaml_from_github(
    "prefect-2-targeting",
    "config/kubernetes/",
    ["node_types.yaml"],
    logging
)
github_node_sizes_retraining

In [25]:
github_node_sizes_df = pd.DataFrame.from_dict(github_node_sizes_retraining["node_types"]["resources"], orient="index").reset_index().rename(columns={"index": "node_type"})
for col in github_node_sizes_df.columns:
    github_node_sizes_df = github_node_sizes_df.rename(columns={col: f"{col}_github"})
github_node_sizes_df

In [26]:
merged_targeting_runs_with_models_with_node_size = pd.merge(
    merged_targeting_runs_with_models_with_node_size,
    github_node_sizes_df,
    left_on="merged_node_name",
    right_on="node_pool_name_github",
    how="left"
)
merged_targeting_runs_with_models_with_node_size[["tenant","timestamp","audience_id","timestamp","targetingOutlookDays","node_name","node_gb_nm","merged_node_name","node_pool_name_github","node_type_github"]]

In [27]:
merged_targeting_runs_with_models_with_node_size["merged_node_name"] = np.where(
    merged_targeting_runs_with_models_with_node_size["merged_node_name"]==merged_targeting_runs_with_models_with_node_size["node_pool_name_github"],
    merged_targeting_runs_with_models_with_node_size["node_type_github"],
    merged_targeting_runs_with_models_with_node_size["merged_node_name"]
)

In [28]:
merged_targeting_runs_with_models_with_node_size[["tenant","timestamp","audience_id","timestamp","targetingOutlookDays","node_name","node_gb_nm","merged_node_name","node_pool_name_github","node_type_github"]]

In [29]:
#merged_targeting_runs_with_models_with_node_size.groupby(by=["tenant","Prefect_Deployments","audience_id"])["merged_node_name"].value_counts().reset_index().sort_values(by="audience_id")

## Check Data Quality of Node Size Mapping

In [30]:
null_node_size_mapping = merged_targeting_runs_with_models_with_node_size[merged_targeting_runs_with_models_with_node_size["merged_node_name"].isnull()].drop_duplicates(subset=["tenant","targetingOutlookDays"])
null_node_size_mapping[["tenant","targetingOutlookDays","date"]]
print(f"Found {len(null_node_size_mapping)} null node size mappings")


In [31]:
null_node_size_mapping[["Prefect_Deployments","tenant","targetingOutlookDays","merged_node_name","audience_id","date","duration","node_name","node_pool_name_github","serviceName"]]#.drop_duplicates()

In [32]:
merged_targeting_runs_with_models_with_node_size.to_csv(f"{path_to_save}extract_targeting_retraining_costs_merged_and_pp.csv", index=False)

In [33]:
merged_targeting_runs_with_models_with_node_size[["tenant","timestamp","audience_id","merged_node_name","duration","charge","part_of_costs","total_charge_of_serviceName"]]

# Extract Valid Time Window for Calculating Costs
- erstmal nur StackIT-Kosten berücksichtigen
- StackIT 03.11 - 10.11: sehr viele Testruns
- stackIT start date: 11.11.2025 (Umzug)
- Zeiträume, wo azure lief (erstmal außen vor lassen): 2024-12-22 & 2025-01-03


In [34]:
stackit_cost_handling

In [35]:
df_cleaned = merged_targeting_runs_with_models_with_node_size[merged_targeting_runs_with_models_with_node_size["date"]> stackit_cost_handling["start"]]
print(f"Min to max date in df_cleaned: {df_cleaned['date'].min()} to {df_cleaned['date'].max()}")
for exlcude_dates in stackit_cost_handling["exlude_date_ranges"]:
    print(f"Excluding date range: {exlcude_dates['start']} to {exlcude_dates['end']}")
    idx_to_remove = df_cleaned[
        (df_cleaned["date"] >= exlcude_dates["start"]) & (df_cleaned["date"] <= exlcude_dates["end"])
    ].index
    df_cleaned = df_cleaned.drop(idx_to_remove)
df_cleaned = df_cleaned.drop(columns=["timestamp_nm"])
df_cleaned=df_cleaned.reset_index(drop=True)
df_cleaned["date"].value_counts().sort_index()

In [36]:
df_cleaned[["Prefect_Deployments","tenant","audience_id","date","duration","node_name","merged_node_name","charge","part_of_costs","total_charge_of_serviceName"]]

In [37]:
df_cleaned["merged_node_name"].sort_values().unique()

In [38]:
count_nodes_by_customer = pd.DataFrame(df_cleaned.groupby(by=["date","tenant","Prefect_Deployments"])["merged_node_name"].unique()).reset_index()
count_nodes_by_customer.sort_values(by=["tenant","date"], ascending=True)

In [39]:
df_cleaned.to_csv(f"{path_to_save}extract_targeting_retraining_costs_cleaned.csv", index=False)

# Get Costs by Service and Date
- grouping cost per cloud
    - azure: via serviceName & date
    - stackit: via serviceName & date
    - aws: via serviceName & date
    - databricks: via serviceName & date

In [51]:
use_columns_node_size_costs = ["cloud","date","node_name","charge","part_of_costs","total_charge_of_serviceName","serviceName"]
data_columns = use_columns_node_size_costs.copy() + ["serviceCategoryName","quantity","billing_currency","machine.type"]
print(df_orignal["date"].astype("string").min(),df_orignal["date"].astype("string").max())
temp_costs = df_orignal.drop_duplicates(subset=use_columns_node_size_costs).reset_index(drop=True)
temp_costs = temp_costs[temp_costs["date"]> stackit_cost_handling["start"]]
temp_costs = temp_costs.dropna(subset=["charge"])
len_data = len(temp_costs)
# handle databricks costs
temp_costs = stack_pp(temp_costs)
temp_costs = handle_aws_costs(temp_costs)
temp_costs = handle_azure_costs(temp_costs)
temp_costs = handle_databricks_cost(temp_costs)
temp_costs = temp_costs[data_columns]
# format data
temp_costs["date"] = temp_costs["date"].astype("string")
temp_costs["date"] = pd.to_datetime(temp_costs["date"], format="mixed", utc=True).dt.strftime("%Y-%m-%d")
temp_costs["date"] = temp_costs["date"].astype("string")
print(temp_costs["date"].min(),temp_costs["date"].max())
if len_data != len(temp_costs): # plus 2 due to agentpools
    raise ValueError("Length of data does not match")
temp_costs.sort_values(by=["date","node_name","serviceName"])

In [52]:
# sum costs over serviceName and same day
temp_costs["sum_costs_serviceName"] = temp_costs.groupby(by=["date","serviceName"])["charge"].transform("sum")
temp_costs["sum_quantity_serviceName"] = temp_costs.groupby(by=["date","serviceName"])["quantity"].transform("sum")
temp_costs = temp_costs.drop_duplicates(subset=["cloud","date","serviceName","node_name","sum_costs_serviceName","sum_quantity_serviceName"])
temp_costs = temp_costs[["date","node_name","serviceName","sum_costs_serviceName","sum_quantity_serviceName","cloud","serviceCategoryName"]]
temp_costs.sort_values(by=["date","cloud","serviceName"])

In [69]:
temp_costs[temp_costs["cloud"]=="aws"]

# Merging Costs and running data

In [70]:
temp_costs_node_name_none = temp_costs[temp_costs["node_name"].isna()]
temp_costs_not_none = temp_costs[temp_costs["node_name"].notna()]
if len(temp_costs_node_name_none) + len(temp_costs_not_none) != len(temp_costs):
    raise ValueError("Length of data does not match")

In [78]:
df_costs = pd.merge(
    df_cleaned,
    temp_costs_not_none,
    how="left",
    left_on=["date","merged_node_name"],
    right_on = ["date","node_name"],
    suffixes=("","_costs")
)
df_costs[["date","Prefect_Deployments","cloud","cloud_costs","account","audience_id","duration","merged_node_name","node_name_costs","serviceName","serviceName_costs","charge","quantity","total_charge_of_serviceName",'sum_costs_serviceName', 'sum_quantity_serviceName']].sort_values(by=["date","merged_node_name","cloud","account","audience_id"])

In [79]:
vc = df_costs.groupby(by=["date","merged_node_name","tenant","audience_id"])["duration"].value_counts().sort_values(ascending=False)
vc = vc[vc>1].reset_index()
if vc.empty is False:
    raise ValueError(f"There are duplicates in the data: {vc}")

# clean data
- fill null cloud values with cloud_costs values
- fill null total_charge_of_serviceName costs with sum_costs_serviceName
- fill null quantiy with sum_quantity_serviceName

In [80]:
df_costs_cleaned = df_costs.copy()
# # print(unequal_costs)
df_costs_cleaned["cloud"] = np.where(df_costs_cleaned["cloud"].isnull(), df_costs_cleaned["cloud_costs"], df_costs_cleaned["cloud"])
df_costs_cleaned["total_charge_of_serviceName"] = np.where(df_costs_cleaned["sum_costs_serviceName"].isnull()==False, df_costs_cleaned["sum_costs_serviceName"], df_costs_cleaned["total_charge_of_serviceName"])
df_costs_cleaned["serviceName"] = np.where(df_costs_cleaned["serviceName"].isnull(), df_costs_cleaned["serviceName_costs"], df_costs_cleaned["serviceName"])
df_costs_cleaned["quantity"] = np.where(df_costs_cleaned["sum_quantity_serviceName"].isnull()==False, df_costs_cleaned["sum_quantity_serviceName"], df_costs_cleaned["quantity"])
df_costs_cleaned["sum_duration_serviceName"] = df_costs_cleaned.groupby(by=["date","serviceName"])["duration"].transform("sum")
df_costs_cleaned["sum_duration_serviceName"] = np.where(
    df_costs_cleaned["serviceName"].str.contains("Purpose Server-g1.3-EU01").fillna(False),
    df_costs_cleaned["sum_duration_serviceName"]+(2*86400), #add agent pools duration
    df_costs_cleaned["sum_duration_serviceName"]
)
df_costs_cleaned = df_costs_cleaned.drop(columns=["cloud_costs","serviceName_costs","sum_costs_serviceName","sum_quantity_serviceName","charge"])
df_costs_cleaned[["date","Prefect_Deployments","cloud","account","audience_id","duration","merged_node_name","node_name_costs","serviceName","quantity","total_charge_of_serviceName","serviceCategoryName"]].sort_values(by=["date","merged_node_name","cloud","account","audience_id"])

In [81]:
#TODO check for resetting the model
accounts = call_api_with_service_token(f"{url}/core/accounts/query", {}, logging)
null_charges = df_costs_cleaned[df_costs_cleaned["total_charge_of_serviceName"].isnull()]
null_charges.shape
if len(null_charges) > 20:
    raise ValueError(f"Found more than {len(null_charges)} null charges")

In [84]:
df_costs_cleaned.to_csv(f"{path_to_save}extract_targeting_retraining_costs_final_{from_date}_{to_date}.csv", index=False)

In [85]:
df_costs_cleaned.columns

In [59]:
# welche kosten gehören noch zum targeing, die nicht die node sizes betreffen?
#TODO: missing azure node_names .. ?

# Check samples of data for a certain date

In [101]:
# Functions for testing
def return_deployment_counts(test_data: pd.DataFrame):
    return test_data["Prefect_Deployments"].value_counts(dropna=False)

In [ ]:
def return_cumulated_costs(test_data: pd.DataFrame, return_part_of_costs_per_run=False):
    cum =  test_data.groupby("serviceName")["cost_per_run"].sum().sort_values(ascending=False)
    cum = pd.DataFrame(cum).reset_index()
    if return_part_of_costs_per_run is True:
        cum_part = test_data.groupby("serviceName")["part_of_costs_per_run"].sum().sort_values(ascending=False)
        cum_part = pd.DataFrame(cum_part).reset_index()
        cum = pd.merge(cum, cum_part, on="serviceName")
    return cum

## 10.06.2025
Prefect Flows:
- expected retrainings: 6
- expected targetings: 114
    - total: 119 - 1 - 4 = 114
    - 1 wurde nach wenigen sekunden gecancelt
    - 4: kein pod wurde gestartet - also keine logs

In [103]:
use_columns = ["date","timestamp","Prefect_Deployments","cloud","account","audience_id","duration","sum_duration_serviceName","merged_node_name","node_name_costs","serviceName","quantity","total_charge_of_serviceName"]
date_20250610 = df_costs_cleaned[df_costs_cleaned["date"] == "2025-06-10"][use_columns]
expected_runs = {"targeting": 114, "retraining": 6, "googleConversionUpdate": 5}
expected_costs_dictionary = {
    "header": ["Service", "SKU", "Gesamtmenge", "Gesamtkosten"],
    "date": "20250610",
    "data": [
        {"Service": "Memory Optimized Server-b1.4-EU01", "SKU": "ST-0020701", "Gesamtmenge": "17 Hours", "Gesamtkosten": "13.73"},
        {"Service": "General Purpose Server-g1.3-EU01", "SKU": "ST-0006901", "Gesamtmenge": "87 Hours", "Gesamtkosten": "13.19"},
        {"Service": "Memory Optimized Server-m1.3-EU01", "SKU": "ST-0009301", "Gesamtmenge": "44 Hours", "Gesamtkosten": "8.47"},
        {"Service": "General Purpose Server-g1.4-EU01", "SKU": "ST-0007301", "Gesamtmenge": "24 Hours", "Gesamtkosten": "7.28"},
        {"Service": "Memory Optimized Server-b1.3-EU01", "SKU": "ST-0020501", "Gesamtmenge": "17 Hours", "Gesamtkosten": "6.86"},
        {"Service": "Compute Optimized Server-c1.3-EU01", "SKU": "ST-0008901", "Gesamtmenge": "43 Hours", "Gesamtkosten": "5.93"},
        {"Service": "Block Storage for disk volumes Premium-Performance 2-EU01", "SKU": "ST-0011001", "Gesamtmenge": "244 Hours", "Gesamtkosten": "4.91"},
        {"Service": "Memory Optimized Server-b1a.16d-EU01", "SKU": "ST-0031001", "Gesamtmenge": "2 Hours", "Gesamtkosten": "4.19"},
        {"Service": "PostgreSQL-Flex-2.4-Single-EU01", "SKU": "ST-0025701", "Gesamtmenge": "24 Hours", "Gesamtkosten": "3.02"},
        {"Service": "Essential-Network-Load-Balancer-10-EU01", "SKU": "ST-0062401", "Gesamtmenge": "211 Hours", "Gesamtkosten": "2.75"},
        {"Service": "Kubernetes Engine-Cluster Management-EU01", "SKU": "ST-0010701", "Gesamtmenge": "24 Hours", "Gesamtkosten": "2.39"},
        {"Service": "Tiny Server-t1.2-EU01", "SKU": "ST-0009101", "Gesamtmenge": "390 Hours", "Gesamtkosten": "2.13"},
        {"Service": "Compute Optimized Server-c1.2-EU01", "SKU": "ST-0008301", "Gesamtmenge": "33 Hours", "Gesamtkosten": "2.00"},
        {"Service": "General Purpose Server-g1.1-EU01", "SKU": "ST-0008501", "Gesamtmenge": "48 Hours", "Gesamtkosten": "1.82"},
        {"Service": "Block Storage for disk volumes Premium-Capacity-EU01", "SKU": "ST-0018001", "Gesamtmenge": "11.598 Gigabyte Hours", "Gesamtkosten": "1.05"},
        {"Service": "Public IP Address (IPv4)-EU01", "SKU": "ST-0070001", "Gesamtmenge": "219 Hours", "Gesamtkosten": "0.89"},
        {"Service": "Block Storage for disk volumes Premium-Performance 1-EU01", "SKU": "ST-0010901", "Gesamtmenge": "72 Hours", "Gesamtkosten": "0.73"},
        {"Service": "Block Storage for PostgreSQL Premium-Performance 2-EU01", "SKU": "ST-0068501", "Gesamtmenge": "24 Hours", "Gesamtkosten": "0.48"},
        {"Service": "Block Storage for PostgreSQL Premium-Capacity-EU01", "SKU": "ST-0069101", "Gesamtmenge": "1.200 Gigabyte Hours", "Gesamtkosten": "0.11"},
        {"Service": "Backup Storage for PostgreSQL Premium-EU01", "SKU": "ST-0040001", "Gesamtmenge": "2.455 Gigabyte Hours", "Gesamtkosten": "0.09"},
        {"Service": "Observability-Metrics-Endpoint-100k-EU01", "SKU": "ST-0101401", "Gesamtmenge": "24 Hours", "Gesamtkosten": "0.00"},
        ]
    }
expected_costs_dictionary.keys()

In [104]:
vc_20250610 = return_deployment_counts(date_20250610)
for key in expected_runs.keys():
    if key not in vc_20250610.index:
        raise ValueError(f"Key {key} not found in {vc_20250610.index}")
    if vc_20250610[key] != expected_runs[key]:
        raise ValueError(f"Expected value {expected_runs[key]} for {key}, but got {vc_20250610[key]}")
vc_20250610

In [105]:
date_20250610 = return_cost_per_run(date_20250610)
date_20250610.sort_values(by=["serviceName","account","audience_id"])

In [106]:
cumulated_costs = return_cumulated_costs(date_20250610, return_part_of_costs_per_run=True)
cumulated_costs

In [107]:
res_20250610 = pd.DataFrame(columns=["serviceName", "cost_per_run", "expected_costs", "diff", "diff_in_perc"])
for cost_entry in expected_costs_dictionary["data"]:
    serviceName = cost_entry["Service"]
    #if "Server" not in serviceName:
    #    continue
    expected_costs = cost_entry["Gesamtkosten"]
    expected_costs = float(expected_costs)
    print(serviceName, expected_costs)
    cumulated_costs_temp = cumulated_costs[cumulated_costs["serviceName"] == serviceName]
    if len(cumulated_costs_temp) != 1:
       print(f"Found {len(cumulated_costs_temp)} entries for serviceName {serviceName}")
       cumulated_costs_temp_value = None
    else:
        cumulated_costs_temp_value = cumulated_costs_temp["cost_per_run"].round(2).values[0]
        if serviceName == "General Purpose Server-g1.3-EU01":
            #special case for agentpools
            known_part = cumulated_costs_temp["part_of_costs_per_run"].values[0]
            total_costs = np.round(cumulated_costs_temp_value / known_part,2)
            print(f"  total costs: {total_costs}")
            print(f"  missing part for agentpools: {total_costs-cumulated_costs_temp_value}")
            cumulated_costs_temp_value = total_costs
    if cumulated_costs_temp_value != expected_costs:
        print(f"Expected {expected_costs} for serviceName {serviceName}, but got {cumulated_costs_temp_value}")
    if cumulated_costs_temp_value is None:
        diff = None
        diff_in_perc = None
    else:
        diff = cumulated_costs_temp_value - expected_costs if cumulated_costs is not None else None
        diff_in_perc = (cumulated_costs_temp_value - expected_costs) / expected_costs
    res_20250610.loc[len(res_20250610)] = [
        serviceName,
        cumulated_costs_temp_value,
        expected_costs,
        diff,
        diff_in_perc
    ]
res_20250610

# OPEN QUESTIONS

General Purpose Server-g1.4-EU01 (8 CPU, 32 GB RAM)
- nicht im k8s node pool
- innkeepr-analytics-2?

Tiny Server-t1.2-EU01 (1 CPU, 1 RAM)
- nicht im k8s node pool
- k8s-svc-7e52454b-48a0-4719-b97c-0d94ca2fb517-ga-conversion-upda -> läuft der, wenn die die core/queue bearbeitet wird?

Compute Optimized Server-c1.2-EU01 (2 CPU, 4 GB RAM)
- k8s: 2cpu4gib
- postgres-prefect (2 CPU, 4GB)

General Purpose Server-g1.1-EU01 (1 CPU, 4 RAM)
- nicht im k8s node pool
- analytics-testing-server

k8s agentpool:
  - agentpool: General Purpose Server-g1.3-EU01 
  - Den sehe ich zweimal in den Servern - für die Region eu01-3 und eu01-2. Laufen die beide 24h?
  - #TODO: Muss herausgerechnet werden / läuft 24h
